Using the fourth representation of Legendre's Polynomial found on this wiki, we can generate the P_n(x) for any value of n. We will then evaluate these polyomials on the domains (-1, 1) to determine vector b and the solve the Vaderman Matrix to determine the weights of the Gaussian Quadrature. 


In [ ]:
import math
import numpy as np
from scipy.optimize import root
from sympy import symbols, integrate, init_printing, Add, pprint, latex, integrate, Poly

from scipy.linalg import lstsq
from tabulate import tabulate


init_printing(use_latex=True)

Currently this generate all of Legendre Polynomials from 0 to n. For the purposes of the project we will need only a function that takes in a symbolic variable and returns the nth Legendre polynial. 


In [ ]:
# Generate n number of Legendre Polynomials of the first kind

# Define the symbolic variable
x = symbols('x')

# Store n legrende polys
Pn = []

max_degree = 20

for n in range(0,max_degree):
    limit = math.floor(n/2)

    terms = []
    # Calculate Terms of Polynomial
    for k in range(0, limit+1):
        term = (1/2**n) * (-1) ** k * math.comb(n,k) * math.comb(2*n - 2*k,n) * x ** (n-2*k)
        terms.append(term)

    # Sum terms to polynomial, pretty print
    Pn.append(Add(*terms))

'''
# Open the file for writing
with open(file_name, 'w') as file:
    # Write LaTeX preamble and begin document
    file.write("\\documentclass{article}\n")
    file.write("\\begin{document}\n")

    # Write each equation in LaTeX format to the file
    for equation in Pn:
        file.write("\\begin{equation}\n")
        file.write(latex(equation) + "\n")
        file.write("\\end{equation}\n")

    # End document
    file.write("\\end{document}\n")
'''

Now that we can generate P_n(x) for any value n and have confirmed that the output is correct we need to evaluate the integral of each equation over the domain (-1,1) in order to construct the nxn matrix. 

Update: The issue with this is that Legendre Polynomials are constructed in the manner that both the function itself and the product of two legendre polynomials of different degrees are orthogonal over the range [-1,1]. This means that the value of the integral should be 0 for n>0. 

Also the legendre(n,x) function from sympy could have been used to generate the legendre polynomials. 

I do not think the following cell is important for what we are exploring, but calculates the integral over the range [0,1] of the Legendre Polynomials that were generate above.


In [ ]:
# Build b vector
degree = 10
b = []

# limits of integration
lower = 0
upper = 1

for i in range(0, degree):
    int_Pn = integrate(Pn[i], (x, lower, upper))
    b.append(int_Pn)

print(b)

The following cell contructs the b vector where each cell is the soltion to the integral on the domain [-1,1] of the function x^n for n > 0 where n is an integer. 


In [ ]:
def generate_b(n):

    lower = -1
    upper = 1

    x = symbols('x')
    b = []

    for i in range(0,n):
        int_xn = integrate(x ** i, (x, lower, upper))
        b.append(int_xn)
    
    return b

Then we will find the roots of the corresponding Legendre Polynomial which we know falls in the range [-1,1] based on the properties of Legendre Polynomials (why exactly this works would be an interesting proof to look into. I think it has something to do with the remainder after polynomial division). 


In [ ]:
def roots_legendre_poly_np(n):
    limit = math.floor(n/2)
    terms = []
    x = symbols('x')

    # Calculate Terms of Polynomial
    for k in range(0, limit+1):
        term = (1/2**n) * (-1) ** k * math.comb(n,k) * math.comb(2*n - 2*k,n) * x ** (n-2*k)
        terms.append(term)

    # Add these terms together into a single polymial (Probably a faster way to do this)
    pn_poly = Poly(sum(terms), x)

    #display(pn_poly)

    pn_coefficients = pn_poly.all_coeffs()

    #print(pn_coefficients)

    pn_roots = np.roots(pn_coefficients)

    return pn_roots

The above code runs into an issue for n > 40 where it incorrectly calculates the root. Keeping this here for demonstration as it is an example of the necessity for better methods of finding roots. Below we take another approach using the scipy.optimize.root function instead. I believe (but need to do more research about the specifics) that this root finding approach using the Jacobi matrix and the QR algorithm for determining roots of polynomials. 

The below code uses the scipy.optimize.root function which is a more robust root solver. This fixes the issue with n> 40. 


In [ ]:
def roots_legendre_poly(n):
    limit = math.floor(n/2)
    terms = []
    x = symbols('x')

    # Calculate Terms of Polynomial
    for k in range(0, limit+1):
        term = (1/2**n) * (-1) ** k * math.comb(n,k) * math.comb(2*n - 2*k,n) * x ** (n-2*k)
        terms.append(term)

    # Add these terms together into a single polynomial
    pn_poly = Poly(sum(terms), x)

    # Extract coefficients
    pn_coefficients = pn_poly.all_coeffs()

    # Define the function for root finding
    def f(x_val):
        return [pn_poly.subs(x, val) for val in x_val]

    # Initial guess for the roots
    initial_guess = np.linspace(-1, 1, n)

    # Find the roots using scipy.optimize.root
    result = root(f, initial_guess, method='hybr')

    pn_roots = result.x

    return pn_roots

Now I need to construct the Vanderman matrix. That is, I need to evaluate each value of the roots (x_1, ..., x_i) for the function x^0, ..., x^n then solve this system of equations for the weights.


In [ ]:
def create_vandermonde(n):
    roots_l = roots_legendre_poly(n)
    vander_matrix = np.vander(roots_l, increasing = True).T
    return vander_matrix, roots_l

At this point we have the full vandermonde matrix evaluated at the Legendre Polynomial roots as well as the monomials of increasing power evaluated at increasing powers. The next step is to solve this system of equations. 


In [ ]:
def calculate_weights(n):  
    A, roots = create_vandermonde(n)
    b = generate_b(n)

    # Assuming generate_b returns a list, convert it to NumPy array
    A = np.array(A)
    b = np.array(b)

    A = A.astype(np.float64)
    b = b.astype(np.float64)

    #w = np.linalg.solve(A,b.T)
    weights, _, _, _ = lstsq(A, b.T)

    #return roots, w
    return {'n': n, 'weights': weights, 'roots': roots}

Check solution by looping from 1 to 10 and printing weights and corresponing roots


In [ ]:
max_n = 6

for i in range(2, max_n + 1):
    result = calculate_weights(i)

    print(f"Table for n = {result['n']}")
    table = {'Weights': result['weights'], 'Roots': result['roots']}
    print(tabulate(table, headers='keys', tablefmt='fancy_grid'))
    print("\n")

Create a Latex Document of the Tables


In [ ]:
max_n = 6
results = []

for i in range(2, max_n + 1):
    result = calculate_weights(i)
    results.append(result)

# Create LaTeX tables
latex_tables = []
for result in results:
    table = {'n': result['n'], 'weights': result['weights'], 'roots': result['roots']}
    latex_tables.append(table)

# Write LaTeX document
latex_document = "\\documentclass{article}\n\\usepackage{amsmath}\n\\begin{document}\n"

for table in latex_tables:
    latex_document += f"\\section*{{Table for n={table['n']}}}\n"
    
    # Create a list of lists (rows) for tabulate
    table_data = [['i', 'Weights', 'Roots']] + list(zip(range(1, len(table['weights']) + 1), table['weights'], table['roots']))
    
    latex_document += tabulate(table_data, headers='firstrow', tablefmt='latex_raw') + "\n\n"

latex_document += "\\end{document}"

# Save LaTeX document to a file
with open('output.tex', 'w') as file:
    file.write(latex_document)

https://engcourses-uofa.ca/books/numericalanalysis/numerical-integration/gauss-quadrature/#introduction2


In [ ]:
|import sympy as sp

sp.init_printing(use_latex=True)

x, x1, x2 = sp.symbols('x x1 x2')
a0, a1, a2, a3 = sp.symbols('a0 a1 a2 a3')
w1, w2 = sp.symbols('w1 w2')

f = a0 + a1*x + a2*x**2 + a3*x**3

I1 = sp.integrate(f, (x, -1, 1))
I2 = w1*(f.subs(x, x1)) + w2*(f.subs(x, x2))

display("I1: ",I1)
display("I2: ",I2)

Eq1 = sp.expand(I1 - I2).coeff(a0)
Eq2 = sp.expand(I1 - I2).coeff(a1)
Eq3 = sp.expand(I1 - I2).coeff(a2)
Eq4 = sp.expand(I1 - I2).coeff(a3)

display("Eq1: ",Eq1)
display("Eq2: ",Eq2)
display("Eq3: ",Eq3)
display("Eq4: ",Eq4)

sol = list(sp.nonlinsolve([Eq1, Eq2, Eq3, Eq4], [x1, x2, w1, w2]))
display(sol)